### Sprint13 ディープラーニングフレームワーク1

コードリーディング

TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。


tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。

In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import statistics
from scipy import stats
import collections
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from tqdm import tqdm
from IPython.display import Image
import tensorflow as tf

### 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。



>重みを初期化する必要があった

>エポックのループが必要だった

>重みの更新

>重みの最適化

>学習率の設定

>バッチサイズの設定

>隠れ層数とノード数の設定


それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

データセットの用意
以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。


目的変数はSpeciesですが、3種類ある中から以下の2種類のみを取り出して使用します。

Iris-versicolor

Iris-virginica

### 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。


《サンプルコード》


＊バージョン1.5から1.14の間で動作を確認済みです。

####  説明は以下のコードにコメントとして加えた。

In [110]:
from sklearn.datasets import load_iris
data = load_iris()
X = data.data
y = data.target
X = X[50:150]
y = y[50:150]-1

In [111]:
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [190]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]  
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [112]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 10
n_hidden1 = 50  #1層目をノード数50に設定
n_hidden2 = 100  #2層目をノード数100に設定
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),#ここから入力から出力層までの重みを設定
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])#ここから初めの層の計算、以降次の層に渡していく。
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5)) #比較して等しい場合 True, 間違いの場合 falseを返す。
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))  #Trueは１, falseは０を返すため、Trueの数をカウントする。それをtf.reduce_meanで平均を出す。
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)  # トータルデータ数をバッチサイズで割り、１エポックあたりのバッチ数算出
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})  # 目的関数の最小化（最適化）を実行
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})# 目的関数、正答率を実行し取得
            total_loss += loss
            total_acc += acc
        total_loss /= total_batch  # 目的関数の結果をデータ数で割り平均算出
        total_acc /= total_batch   # 正解率の結果をデータ数で割り平均算出
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 56.7000, val_loss : 25.4921, acc : 0.250, val_acc : 0.625
Epoch 1, loss : 2.8369, val_loss : 0.5100, acc : 0.500, val_acc : 0.875
Epoch 2, loss : 7.2071, val_loss : 10.7963, acc : 0.750, val_acc : 0.375
Epoch 3, loss : 2.8267, val_loss : 2.5766, acc : 0.750, val_acc : 0.625
Epoch 4, loss : 3.4538, val_loss : 1.4914, acc : 0.500, val_acc : 0.750
Epoch 5, loss : 0.0243, val_loss : 0.0346, acc : 1.000, val_acc : 1.000
Epoch 6, loss : 0.1990, val_loss : 0.0919, acc : 0.750, val_acc : 0.938
Epoch 7, loss : 0.0005, val_loss : 0.0323, acc : 1.000, val_acc : 1.000
Epoch 8, loss : 0.0017, val_loss : 0.2206, acc : 1.000, val_acc : 0.938
Epoch 9, loss : 0.0001, val_loss : 0.0441, acc : 1.000, val_acc : 1.000
test_acc : 0.850


他のデータセットへの適用

これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。


Iris（3種類全ての目的変数を使用）
House Prices

どのデータセットもtrain, val, testの3種類に分けて使用してください。

### 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。


Iris Species


2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。


《ヒント》


以下の2箇所は2クラス分類特有の処理です。


loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))


In [197]:
from sklearn.datasets import load_iris
data = load_iris()
X = data.data
y = data.target
X = X[0:150]
y = y[0:150]
y = pd.get_dummies(y).values

In [198]:
#y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [200]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを３値分類する
"""


# ハイパーパラメータの設定
learning_rate = 0.1
batch_size = 10
num_epochs = 10
n_hidden1 = 100  #1層目をノード数50に設定
n_hidden2 = 50  #2層目をノード数100に設定
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3 #クラス数を3に変更
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),#ここから入力から出力層までの重みを設定
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])#ここから初めの層の計算、以降次の層に渡していく。
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    #layer_output = tf.nn.softmax(layer_output)
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
#loss_op = - tf.reduce_mean(tf.reduce_sum(tf.log(logits+0.0001) * Y, axis=1),axis=0)  #tf.reduce_mean((labels=Y, logits=logits)) #ソフトマックスに変更
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))  #正解とソフトマックス結果の最も確率の高いラベルを比較しBool値を返す。
#correct_pred = tf.equal(tf.dtypes.cast(tf.argmax(Y, 1), tf.int32), tf.dtypes.cast(tf.argmax(logits, 1), tf.int32))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) #cast()で　trueの成果のみ１、Falseは０を出力
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)  # トータルデータ数をバッチサイズで割り、１エポックあたりのバッチ数算出
        total_loss = 0
        total_acc = 0
        total_val_loss = 0
        total_val_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            #sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})  # 目的関数の最小化（最適化）を実行
            _ , loss, acc = sess.run([train_op, loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})# 目的関数、正答率を実行し取得
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
            total_loss += loss
            total_acc += acc
            total_val_loss += val_loss
            total_val_acc += val_acc
            
        total_loss /= total_batch  # 目的関数の結果をデータ数で割り平均算出
        total_acc /= total_batch   # 正解率の結果をデータ数で割り平均算出
        #val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, total_val_loss, total_acc, total_val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 134.6390, val_loss : 1157.7435, acc : 0.467, val_acc : 5.042
Epoch 1, loss : 15.4695, val_loss : 65.6887, acc : 0.743, val_acc : 7.792
Epoch 2, loss : 0.7215, val_loss : 10.6150, acc : 0.933, val_acc : 8.583
Epoch 3, loss : 2.4999, val_loss : 27.3204, acc : 0.800, val_acc : 7.875
Epoch 4, loss : 0.9119, val_loss : 13.8477, acc : 0.933, val_acc : 9.083
Epoch 5, loss : 0.2336, val_loss : 8.3823, acc : 0.970, val_acc : 9.250
Epoch 6, loss : 0.3329, val_loss : 7.3594, acc : 0.943, val_acc : 8.875
Epoch 7, loss : 0.0681, val_loss : 8.0416, acc : 0.970, val_acc : 9.250
Epoch 8, loss : 0.2395, val_loss : 13.5195, acc : 0.960, val_acc : 8.875
Epoch 9, loss : 0.9207, val_loss : 21.7697, acc : 0.920, val_acc : 8.500
test_acc : 0.967


### 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。

In [163]:
df = pd.read_csv('train.csv')
df.head()
X = df[["GrLivArea","YearBuilt"]].values
y = df["SalePrice"].values
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_val = scaler.transform(y_val)
y_test = scaler.transform(y_test)

In [165]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 10
n_hidden1 = 100  #1層目をノード数50に設定
n_hidden2 = 50  #2層目をノード数100に設定
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),#ここから入力から出力層までの重みを設定
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])#ここから初めの層の計算、以降次の層に渡していく。
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_sum(tf.square(Y - logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

# 指標値計算
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)  # トータルデータ数をバッチサイズで割り、１エポックあたりのバッチ数算出
        total_loss = 0
        total_acc = 0
        total_val_loss = 0
        
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            #sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})  # 目的関数の最小化（最適化）を実行
            _ , loss = sess.run([train_op,loss_op], feed_dict={X: mini_batch_x, Y: mini_batch_y})# 目的関数、正答率を実行し取得
            val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
            total_loss += loss
            total_val_loss += val_loss
            #total_acc += acc
        total_loss /= total_batch  # 目的関数の結果をデータ数で割り平均算出
        #total_acc /= n_samples   # 正解率の結果をデータ数で割り平均算出
        #val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        total_val_loss /= total_batch
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, total_loss, total_val_loss))
    test_loss = sess.run(loss_op, feed_dict={X: X_test, Y: y_test})
    print("test_loss : {:.3f}".format(test_loss))

Epoch 0, loss : 5107.3074, val_loss : 117734.9174
Epoch 1, loss : 326.0724, val_loss : 6566.6058
Epoch 2, loss : 115.2316, val_loss : 2080.3622
Epoch 3, loss : 62.2007, val_loss : 1151.1883
Epoch 4, loss : 42.9790, val_loss : 864.4940
Epoch 5, loss : 34.3342, val_loss : 742.6349
Epoch 6, loss : 29.7463, val_loss : 659.8783
Epoch 7, loss : 26.7846, val_loss : 609.3767
Epoch 8, loss : 24.1758, val_loss : 579.8185
Epoch 9, loss : 21.9599, val_loss : 543.0723
test_loss : 732.596


### 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

In [206]:
# データロード
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# channel次元の追加(NCHW)
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# 正規化
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

#分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

# one_hot化
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_val = enc.fit_transform(y_val[:, np.newaxis])
y_test = enc.transform(y_test[:, np.newaxis])

In [207]:
"""
TensorFlowで実装したニューラルネットワークを使いMNISTデータセットを10値分類する
"""


# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 20
num_epochs = 10
n_hidden1 = 50  #1層目をノード数50に設定
n_hidden2 = 100  #2層目をノード数100に設定
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10 #クラス数を10に変更
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),#ここから入力から出力層までの重みを設定
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])#ここから初めの層の計算、以降次の層に渡していく。
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    #layer_output = tf.nn.softmax(layer_output)
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
#loss_op = - tf.reduce_sum(tf.reduce_sum(tf.log(logits) * Y, axis=1),axis=0)  #tf.reduce_mean((labels=Y, logits=logits)) #ソフトマックスに変更
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))  #正解とソフトマックス結果の最も確率の高いラベルを比較しBool値を返す。
#correct_pred = tf.equal(tf.dtypes.cast(tf.argmax(Y, 1), tf.int32), tf.dtypes.cast(tf.argmax(logits, 1), tf.int32))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) #cast()で　trueの成果のみ１、Falseは０を出力
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)  # トータルデータ数をバッチサイズで割り、１エポックあたりのバッチ数算出
        total_loss = 0
        total_acc = 0
        total_val_loss = 0
        total_val_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            #sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})  # 目的関数の最小化（最適化）を実行
            _ , loss, acc = sess.run([train_op, loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})# 目的関数、正答率を実行し取得
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
            total_loss += loss
            total_acc += acc
            total_val_loss += val_loss
            total_val_acc += val_acc
        total_loss /= total_batch  # 目的関数の結果をデータ数で割り平均算出
        total_acc /= total_batch   # 正解率の結果をデータ数で割り平均算出
        total_val_loss /= total_batch  # 目的関数の結果をデータ数で割り平均算出
        total_val_acc /= total_batch   # 正解率の結果をデータ数で割り平均算出
        #val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, total_val_loss, total_acc, total_val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 40.5137, val_loss : 40.5637, acc : 0.635, val_acc : 0.632
Epoch 1, loss : 7.5872, val_loss : 8.0534, acc : 0.792, val_acc : 0.782
Epoch 2, loss : 4.1835, val_loss : 4.7173, acc : 0.822, val_acc : 0.810
Epoch 3, loss : 2.7153, val_loss : 3.3178, acc : 0.845, val_acc : 0.827
Epoch 4, loss : 1.9362, val_loss : 2.5850, acc : 0.862, val_acc : 0.839
Epoch 5, loss : 1.4931, val_loss : 2.1718, acc : 0.873, val_acc : 0.849
Epoch 6, loss : 1.1974, val_loss : 1.8893, acc : 0.886, val_acc : 0.857
Epoch 7, loss : 0.9851, val_loss : 1.6727, acc : 0.894, val_acc : 0.863
Epoch 8, loss : 0.8400, val_loss : 1.5269, acc : 0.900, val_acc : 0.869
Epoch 9, loss : 0.7211, val_loss : 1.4242, acc : 0.909, val_acc : 0.874
test_acc : 0.887
